# What am I listening on Spotify?

In [11]:
import requests
import json
import pandas as pd
import numpy as np
import plotly.express as px

from datetime import datetime

In [27]:
df_streaming = pd.read_json('MyData/StreamingHistory0.json')

df_track = pd.read_json('MyData/tracks.json')

In [28]:
df_streaming['UniqueID'] = df_streaming['artistName'] + ':' + df_streaming['trackName']
df_streaming.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-09-16 22:59,Mark Maxwell,Sweetness,199850,Mark Maxwell:Sweetness
1,2020-09-17 13:37,Anne Reburn,The Longest Time,15298,Anne Reburn:The Longest Time
2,2020-09-21 20:07,Donny Osmond,"I'll Make a Man Out of You - From ""Mulan""/Soun...",83935,Donny Osmond:I'll Make a Man Out of You - From...
3,2020-09-23 12:48,Gorillaz,Strange Timez (feat. Robert Smith),29937,Gorillaz:Strange Timez (feat. Robert Smith)
4,2020-09-23 12:49,Welcome to Night Vale,New to Night Vale? Start here!,66938,Welcome to Night Vale:New to Night Vale? Start...


In [4]:
df_streaming.describe(include = "all")

,endTime,artistName,trackName,msPlayed,UniqueID
count,3436,3436,3436,3.436000e+03,3436
unique,2021,562,1116,NaN,1125
top,2021-06-20 22:11,BRADIO,幸せのシャナナ,NaN,BRADIO:幸せのシャナナ
freq,100,279,22,NaN,22
mean,NaN,NaN,NaN,1.088752e+05,NaN
std,NaN,NaN,NaN,1.689687e+05,NaN
min,NaN,NaN,NaN,0.000000e+00,NaN
25%,NaN,NaN,NaN,2.557750e+03,NaN
50%,NaN,NaN,NaN,3.089350e+04,NaN
75%,NaN,NaN,NaN,2.044170e+05,NaN


In [29]:
df_track['UniqueID'] = df_track['artist'] + ':' + df_track['track']
df_track['track:uri'] = df_track['uri'].str.split('spotify:').str[1]
df_track.head()

,artist,album,track,uri,UniqueID,track:uri
0,Panic! At The Disco,"Too Weird to Live, Too Rare to Die!",Girls / Girls / Boys,spotify:track:2wEjPgOCEczmjlvq60LyQi,Panic! At The Disco:Girls / Girls / Boys,track:2wEjPgOCEczmjlvq60LyQi
1,BRADIO,Flyers,Flyers,spotify:track:52sk8TPaHcLQVCjFaeRI6t,BRADIO:Flyers,track:52sk8TPaHcLQVCjFaeRI6t
2,RADWIMPS,君の名は。,三葉のテーマ,spotify:track:5zEKHWr92eqOaoCgmELP8k,RADWIMPS:三葉のテーマ,track:5zEKHWr92eqOaoCgmELP8k
3,Hello Sleepwalkers,Masked Monkey Awakening,Tenchi Souzou,spotify:track:4PGluOFvKJx9kTtAbOeVJl,Hello Sleepwalkers:Tenchi Souzou,track:4PGluOFvKJx9kTtAbOeVJl
4,Rainbow Dash,It's a Pony Kind of Christmas,Jingle Bells,spotify:track:2Y42Fkup0CVfrNtxNUdz5f,Rainbow Dash:Jingle Bells,track:2Y42Fkup0CVfrNtxNUdz5f


In [6]:
df_track.describe()

,artist,album,track,uri,UniqueID,track:uri
count,385,385,385,385,385,385
unique,66,55,385,385,385,385
top,Martin Baltscheit,A Million Ways to Die in the West,Girls / Girls / Boys,spotify:track:2wEjPgOCEczmjlvq60LyQi,Panic! At The Disco:Girls / Girls / Boys,track:2wEjPgOCEczmjlvq60LyQi
freq,57,57,1,1,1,1


In [30]:
df_merged = pd.merge(df_streaming, df_track[["album", "UniqueID", "track:uri"]], how = "left", on = ["UniqueID"])

Let's see what 

In [15]:
df_merged.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album,track:uri
0,2020-09-16 22:59,Mark Maxwell,Sweetness,199850,Mark Maxwell:Sweetness,NaN,NaN
1,2020-09-17 13:37,Anne Reburn,The Longest Time,15298,Anne Reburn:The Longest Time,The Longest Time,track:4gUTsJRbx7hlU5ocgP7HX9
2,2020-09-21 20:07,Donny Osmond,"I'll Make a Man Out of You - From ""Mulan""/Soun...",83935,Donny Osmond:I'll Make a Man Out of You - From...,NaN,NaN
3,2020-09-23 12:48,Gorillaz,Strange Timez (feat. Robert Smith),29937,Gorillaz:Strange Timez (feat. Robert Smith),NaN,NaN
4,2020-09-23 12:49,Welcome to Night Vale,New to Night Vale? Start here!,66938,Welcome to Night Vale:New to Night Vale? Start...,NaN,NaN


In [36]:
df_merged["HoursPlayed"] = (df_merged["msPlayed"]/(1000*60*60))%24

In [62]:
grouped_artists = df_merged.groupby("artistName")
artists = df_merged["artistName"].unique()
dummy_list =  []
for artist in artists:
    if grouped_artists.get_group(artist)["HoursPlayed"].sum() > 0.5:
        dummy_list.append(grouped_artists.get_group(artist))
    
df_merged_copy = pd.concat(dummy_list, ignore_index=True)                              

In [61]:
fig = px.histogram(df_merged_copy, x = "artistName", y = "HoursPlayed")
fig.show()